# 参考资料

- [python登录豆瓣并获取自己看过的电影 - Johu - 博客园](https://www.cnblogs.com/johu/p/12970233.html)
- [豆瓣Cookie的作用及正确获取方式 | dboke.com](https://blog.200300.xyz/7e49244edf5348b58b91e7c36e37f1ce)

# 导入相关的包

In [ ]:
import requests
from pyquery import PyQuery as pq 
import re
import csv
from fake_useragent import UserAgent
import pandas as pd

# 构造url

In [2]:
df = pd.read_csv("../data/douban_contacts.csv")

In [3]:
user_id_list = df.user_id.to_list()

In [4]:
user_id_list[0]

'atreides1030'

In [5]:
book_read_url = "https://book.douban.com/people/{user}/collect?start={num}&sort=time&rating=all&filter=all&mode=grid"
#某用户已读书籍首页
book_read_start_url = "https://book.douban.com/people/{user}/collect?start=0&sort=time&rating=all&filter=all&mode=grid"


In [ ]:
# 判断一共有多少页

def get_total_page(url):
    headers = {
        'User-Agent': UserAgent().random
    }
    response = requests.get(url, headers=headers)
    html = response.text
    # 使用 PyQuery 解析 HTML
    doc = pq(html)
    # 选择 div.paginator 下的最后一个 a 标签，并获取其文本内容
    total_page = doc('.paginator a:last').text()  
    return int(total_page)

In [7]:
test_url = book_read_url.format(user=user_id_list[0],num="0")

test_url

'https://book.douban.com/people/atreides1030/collect?start=0&sort=time&rating=all&filter=all&mode=grid'

In [37]:
test_url2 = "https://book.douban.com"

# 请求网页

In [ ]:
from selenium import webdriver

browser = webdriver.Firefox()
browser.get('https://www.douban.com/')

# 手动登录豆瓣

# 获取 Cookies
selenium_cookies = browser.get_cookies()

browser.quit()

# 打印 Cookies 或保存
for cookie in selenium_cookies:
    print(cookie)
    

import requests


# 转换为 Requests 可用的 Cookies
requests_cookies = {cookie['name']: cookie['value'] for cookie in selenium_cookies}

# 使用 Requests 发送带 Cookies 的请求
session = requests.Session()
session.cookies.update(requests_cookies)

# 访问目标页面
# 尝试加入user-agent
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
            AppleWebKit/537.36 (KHTML, like Gecko) \
           Chrome/119.0.0.0 Safari/537.36 Edg/119.0.0.0'}


response = session.get("https://www.douban.com/people/dqu",headers=headers)
print(response.text)


<Response [403]>

# 解析网页

In [ ]:
# 函数：解析数据  extract_bookinfo_list(html)
# 参数说明：html为网页源代码的字符串
# 返回值：bookinfo_list为书籍的字典列表
def extract_bookinfo_list(html):         # 添加：定义函数名
    bookinfo_list = []
    doc = pq(html)
    for book in doc.items('div.info'):
        book_name = book('h2 a').text() #书名   
        comments = book('div.short-note p').text() #短评
        rating_raw = book('div.short-note div span:first').attr("class") #评分
        rating = re.findall('\d',rating_raw)
        info_list = book('div.pub').text().split('/')   #书籍信息 
        publisher = info_list[-3].strip()   
        pub_time = info_list[-2]
        price = info_list[-1]
        authors = ''.join(info_list[:-3])
        date_raw = book('span.date').text()       
        date = re.findall('[0-9]+-[0-9]+-[0-9]+', date_raw)[0]

        bookinfo = {'book_name':book_name,
                    'authors':authors,
                    'publisher':publisher,
                    'pub_time':pub_time,
                    'comments':comments,
                    'rating':rating,
                    'date':date,
                    'price':price,
                    }

        bookinfo_list.append(bookinfo)

        
    return bookinfo_list               # 添加：返回书籍的字典列表

In [ ]:
doc = pq(test_html)
doc

for book in doc.items("div.info"):
    print(book("h2 a").text())

In [17]:
extract_bookinfo_list(test_html)

[]